In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain_groq import ChatGroq

In [11]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


llm=ChatGroq(model="gemma2-9b-it")
#llm = ChatOpenAI(model="gpt-4o")
result=llm.invoke("Hello")
result

AIMessage(content='Hello! 👋\n\nHow can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 10, 'total_tokens': 24, 'completion_time': 0.025454545, 'prompt_time': 0.001169269, 'queue_time': 0.188290984, 'total_time': 0.026623814}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6f8fa6ed-8ccf-4860-a84b-70cd9dbd0f18-0', usage_metadata={'input_tokens': 10, 'output_tokens': 14, 'total_tokens': 24})

In [16]:
from pydantic import BaseModel,Field

class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback for the essay")
    score: int = Field(description="score out of 10",ge=0,le=10)

In [17]:
model = llm.with_structured_output(EvaluationSchema)

In [14]:
essay = """
This essay addresses the topic of internet safety in terms of how it relates to school contexts. A focus is taken on compulsory education within the UK. The essay draws on governmental resources, and from work from charitable and educational sector organisations, in supporting online safety measures relevant to schools. The essay also discusses the implications of the 2018 GDPR (General Data Protection Regulations) in terms of their relevance for educational settings.

With the pace of developments in online technology, the relationship between the operation and administration of education and convergence culture bringing the online and the offline together in ever more-complex ways, and the increasing sophistication of the uses made of technology, it is imperative that schools keep abreast of current thinking regarding internet safety (Jenkins, 2008; Hunter, 2012). Statutory guidance from the UK government with respect to schools’ safety is updated regularly; the next iteration of this guidance becomes effective September 2018, replacing the current 2016 version (Department for Education, 2018; Department for Education, 2016). The guidance – entitled Keeping Children Safe in Education – discusses recruitment checks, safeguarding protocols and how to work in instances where allegations are made; an annex to the document focuses on online safety (Department for Education, 2016). Here, the potential issues which face educationalists and learners alike are summarised: these range from online radicalisation, accessing illegal or pornographic material, child sexual exploitation and the activities of sexual predators, both within and outside the school (Department for Education, 2016).

The guidance document advocates a whole-school approach in terms of three main areas of risk and concern (Department for Education, 2016). First, that of content: “being exposed to illegal, inappropriate or harmful material [including] fake news, racist or radical and extremist news” (Department for Education, 2018, p. 92). The second area is that of contact; this is outlined not only in terms of abusive or predatory behaviour by others, but also commercial advertising. The third area is conceived of as conduct-related: “personal online behaviour that increases the likelihood of, or causes, harm; for example making, sending and receiving explicit images, or online bullying” (Department for Education, 2018, p. 92).        

An ongoing requirement is for settings to limit pupils’ access to potential risk via the school’s computer system through the use of appropriate filtering software and oversight via monitoring systems; while there is latitude on the nature of the systems and policies put into place so that the school can contextualise their approach to local needs, there is an expectation that settings will articulate their online safety protocols with their Prevent duty risk assessment (Department for Education, 2018; HM Government, 2016). The Prevent duty, which explains public bodies’ obligations under the Counter-Terrorism and Security Act 2015 to support prevention of people being drawn towards terrorism and other forms of extremist activity, states that such bodies are “expected to ensure children are safe from terrorist and extremist material when accessing the internet in school, including by establishing appropriate levels of filtering” (HM Government, 2016, p. 12; HM Government, 2015). Furthermore, this should be supported by staff training to support school staff in identifying such risks, challenging extremist thought, and making appropriate referrals where there are concerns to be addressed (HM Government, 2016). 

Filtering software should be flexible to that it can be adjusted - for age groups or other forms of differentiation where appropriate - should be easily-controllable by staff, be backed by a clear policy, and be able to identify users. Such software should operate at a network level rather than at the level of the individual device being used, and should allow reporting where problematic sites or other issues are encountered so that both system usage and new sites where there are concerns may be addressed (UK Safer Internet Centre, 2018). Furthermore, there should be an interlocking range of monitoring strategies in place. These include physical monitoring of learner online activity by staff, oversight of search terms and of sites accessed, with the capacity for internet access to be suspended immediately is an issue is encountered, and the issue of technological solutions which may, for instance, be keyword or keystroke-responsive (UK Safer Internet Centre, 2018).
"""

In [21]:
prompt = f"Evaluate the given essay and score it on basis of language quality, clearity of thought and provide detailed feedback and a score out of 10\n{essay}"
model.invoke(prompt)

EvaluationSchema(feedback='The essay provides a comprehensive overview of internet safety in school contexts, drawing on relevant governmental and organizational resources. It effectively highlights the key risks and concerns, including online radicalisation, access to harmful content, and cyberbullying. The essay demonstrates a good understanding of the legal framework surrounding online safety, particularly the GDPR and the Prevent duty. The structure is clear and well-organized, with a logical progression of ideas. However, the language could be more concise and engaging. There is also an opportunity to provide more specific examples and case studies to illustrate the points made.', score=7)

In [24]:
import operator
from typing import Annotated,TypedDict
from langgraph.graph import START,END, StateGraph

class ScoreState(TypedDict):
    essay: str
    language_feedback : str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_feedback: str
    individual_scores : Annotated[list[int],operator.add]
    avg_score : float

In [ ]:
# Declaring Nodes

def clarity(ScoreState):
    prompt = f"Evaluate the given essay and score it on basis of clarity of thought and score it out of 10\n{essay}"
    res = model.invoke(prompt)
    return {"clarity_feedback": res.feedback, individual_score: res.score}

def language(ScoreState):
    prompt = f"Evaluate the given essay and score it on basis of the language and grammer correctness and score it out of 10\n{essay}"
    res = model.invoke(prompt)
    return {"language_feedback": res.feedback,individual_score: res.score}

def analyis(ScoreState):
    prompt = f"Evaluate the given essay and score it on basis of the depth of analysis about the topic and score it out of 10\n{essay}"
    res = model.invoke(prompt)
    return {"language_feedback": res.feedback,individual_score: res.score}

def final_evaluation(ScoreState):
    prompt = f"Evaluate the given essay in general and overall and score it out of 10\n{essay}"
    res = model.invoke(prompt)
    return {"language_feedback": res.feedback,individual_score: res.score}


In [ ]:
graph = StateGraph(ScoreState)

graph.add_node("clarity",clarity)
graph.add_node("language",language)
graph.add_node("analysis",analyis)

